# Additional End of week Exercise - week 2

Now use everything you've learned from Week 2 to build a full prototype for the technical question/answerer you built in Week 1 Exercise.

This should include a Gradio UI, streaming, use of the system prompt to add expertise, and the ability to switch between models. Bonus points if you can demonstrate use of a tool!

If you feel bold, see if you can add audio input so you can talk to it, and have it respond with audio. ChatGPT or Claude can help you, or email me if you have questions.

I will publish a full solution here soon - unless someone beats me to it...

There are so many commercial applications for this, from a language tutor, to a company onboarding solution, to a companion AI to a course (like this one!) I can't wait to see your results.

## My Tutor text board
* ask gpt-40-mini a python math problem, then use my tool to do that  

In [23]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI
# import ollama
import gradio as gr


In [24]:
# constants

MODEL = 'gpt-4o-mini'
# MODEL_LLAMA = 'llama3.2'

In [25]:
# set up environment
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
openai = OpenAI()

API key looks good so far


In [42]:
system_message = "You are a helpful technical assistant. If asked of online course prices, then provide.  "
system_message += "Give concise and informative answers, try no more than 5 sentences. "
system_message += "Always be accurate. If you don't know the answer, say so."

In [86]:
# Let's start by making a useful function

course_price = {"ai": "$119", "python": "$49", "machine learning": "$99", "llm": "$199"}

def get_course_price(online_course):
    print(f"-> Tool: get_course_price is called for course => {online_course}\n")
    course = online_course.lower()
    return course_price.get(course, "Unknown")

In [87]:
get_course_price("LLM")

-> Tool: get_course_price is called for course => LLM



'$199'

In [81]:
# There's a particular dictionary structure that's required to describe our function:

price_function = {
    "name": "get_course_price",
    "description": "Get the price of the price for an online course. Call this whenever you need to know the course price, for example when a customer asks 'How much is the course for LLM'",
    "parameters": {
        "type": "object",
        "properties": {
            "online_course": {
                "type": "string",
                "description": "The online course that the customer wants to take",
            },
        },
        "required": ["online_course"],
        "additionalProperties": False
    }
}

In [82]:
tools = [{"type": "function", "function": price_function}]

In [83]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    # print("response ->", response, "\n")

    # The description and parameters you provide act like an internal “map” that helps 
    # the model match intent to function.
    if response.choices[0].finish_reason=="tool_calls":  # it's the assistant making the tool call
        message = response.choices[0].message
        # print("message ->", message, "\n")
        
        response, course = handle_tool_call(message)
        # print("response -> ", response , "\n", course, "\n")
        messages.append(message)
        messages.append(response)
        response = openai.chat.completions.create(model=MODEL, messages=messages)
    
    return response.choices[0].message.content

In [84]:
# We have to write that function handle_tool_call:

def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    course = arguments.get('online_course')
    price = get_course_price(course)
    response = {
        "role": "tool",
        "content": json.dumps({"online_course": course,"price": price}),
        "tool_call_id": tool_call.id
    }
    return response, course

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()

In [16]:
# here is the question; type over this to ask something new

# question = """
# Please explain what this code does and why:
# yield from {book.get("author") for book in books if book.get("author")}
# """
# question = 'how does AI tool cursor work, be super brief please?'
# question = 'how does git work, please be super brief? please translate to chinese.'

# question = input("Enter your question: ")


In [ ]:
# This function looks rather simpler than the one from my video, because we're taking advantage of the latest Gradio updates

def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages)
    return response.choices[0].message.content

gr.ChatInterface(fn=chat, type="messages").launch()

In [17]:
# prompts

system_prompt = "You are a helpful technical tutor who answers questions about python code, software engineering, data science and LLMs"
user_prompt = "Please give a detailed explanation to the following question: " + question

In [18]:
user_prompt

'Please give a detailed explanation to the following question: how does AI tool cursor work, be super brief please?'

In [19]:
messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
          ]

In [20]:
# Get gpt-4o-mini to answer, w/o streaming
def tutor():
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
          ]
    )
    result = response.choices[0].message.content
    display(Markdown(result))
    
tutor()


The AI tool "Cursor" works primarily as an integrated development environment (IDE) enhanced by artificial intelligence capabilities. It assists developers by providing features such as code suggestions, autocomplete, error detection, and real-time documentation. 

Here's a brief breakdown of its operation:

1. **Context Awareness**: Cursor analyzes the context of the code being worked on, including the current file, libraries in use, and coding patterns.

2. **AI Models**: It employs natural language processing and machine learning models trained on extensive codebases to understand coding syntax and semantics.

3. **Code Suggestions**: As developers write code, Cursor predicts and suggests completions or corrections based on the learned patterns.

4. **Integration**: The tool integrates with popular coding platforms and languages, allowing seamless use within the developer's workflow.

5. **Learning and Adaptation**: Cursor may refine its suggestions as it learns from the user's coding style and preferences over time.

Overall, Cursor aims to enhance productivity by making coding faster and more intuitive.